In [1]:
import pandas as pd # read the csv
import re # regex to detect username, url, html entity 
import nltk # to use word tokenize (split the sentence into words)
import numpy as np
from nltk.corpus import stopwords # to remove the stopwords
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from keras.utils import to_categorical
from keras import backend as K

In [2]:
# initiate the path and read it
dataset_path = "./labeled_data.csv"
df = pd.read_csv(dataset_path)
df.head()

# dataset shape to know how many tweets in the datasets
print(f"num of tweets: {df.shape}")

# extract the text and labels
tweet = list(df['tweet'])
labels = list(df['class'])

num of tweets: (24783, 7)


### Steps:
1. preprocessing
2. splitting
3. tokenize & padding
4. Create model & train
5. evaluate

---

Preprocessing (cleaning the datasets):
- remove html entity
- change user tags (@xxx -> user)
- remove urls
- remove unnecessary  symbol ('', !, ", ') -> cause a lot of noise in the dataset
- remove stopwords

In [3]:
## notes: all of the function taking 1 text at a time
stop_words = set(stopwords.words('english'))
# add rt to remove retweet in dataset (noise)
stop_words.add("rt")

# remove html entity:
def remove_entity(raw_text):
    entity_regex = r"&[^\s;]+;"
    text = re.sub(entity_regex, "", raw_text)
    return text

# change the user tags
def change_user(raw_text):
    regex = r"@([^ ]+)"
    text = re.sub(regex, "user", raw_text)

    return text

# remove urls
def remove_url(raw_text):
    url_regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    text = re.sub(url_regex, '', raw_text)

    return text

# remove unnecessary symbols
def remove_noise_symbols(raw_text):
    text = raw_text.replace('"', '')
    text = text.replace("'", '')
    text = text.replace("!", '')
    text = text.replace("`", '')
    text = text.replace("..", '')

    return text

# remove stopwords
def remove_stopwords(raw_text):
    tokenize = nltk.word_tokenize(raw_text)
    text = [word for word in tokenize if not word.lower() in stop_words]
    text = " ".join(text)

    return text

## this function in to clean all the dataset by utilizing all the function above
def preprocess(datas):
    clean = []
    # change the @xxx into "user"
    clean = [change_user(text) for text in datas]
    # remove emojis (specifically unicode emojis)
    clean = [remove_entity(text) for text in clean]
    # remove urls
    clean = [remove_url(text) for text in clean]
    # remove trailing stuff
    clean = [remove_noise_symbols(text) for text in clean]
    # remove stopwords
    clean = [remove_stopwords(text) for text in clean]

    return clean

In [4]:
# call the cleaning function
clean_tweet = preprocess(tweet)

Splitting the dataset into test and validation

In [5]:
X_train, X_test, y_train, y_test = train_test_split(clean_tweet, labels, test_size=0.2, random_state=42)

Tokenizing and Padding

In [6]:
## Tokenizing -> basically we use tokenisation for many things, its commonly used for feature extraction in preprocessing. btw idk how it works as feature extraction tho :(
# declare the tokenizer
tokenizer = Tokenizer()
# build the vocabulary based on train dataset
tokenizer.fit_on_texts(X_train)
# tokenize the train and test dataset
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# vocabulary size (num of unique words) -> will be used in embedding layer
vocab_size = len(tokenizer.word_index) + 1

In [7]:
## Padding -> to uniform the datas
max_length = max(len(seq) for seq in X_train)

# to test an outlier case (if one of the test dataset has longer length)
for x in X_test:
    if len(x) > max_length:
        print(f"an outlier detected: {x}")

X_train = pad_sequences(X_train, maxlen = max_length)
X_test = pad_sequences(X_test, maxlen = max_length)

In [8]:
# create hot_labels (idk whty tapi ini penting)
y_test = to_categorical(y_test, num_classes=3)
y_train = to_categorical(y_train, num_classes=3)

In [9]:
# another look on the number of tweet in test and training data
print(f"num test tweet: {y_test.shape[0]}")
print(f"num train tweet: {y_train.shape[0]}")

num test tweet: 4957
num train tweet: 19826


Model building

In [10]:
# use some early stopping to get the right epoch
# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# evaluation metrics recall, precision, and f1
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    precisions = precision(y_true, y_pred)
    recalls = recall(y_true, y_pred)
    return 2*((precisions*recalls)/(precisions+recalls+K.epsilon()))

In [11]:
# change dis if u want
output_dim = 100

# LSTM model architechture (CNN + LSTM)
model = Sequential([
    # embedding layer is like idk
    Embedding(vocab_size, output_dim, input_length=max_length),
    # lstm for xxx
    LSTM(64, dropout=0.3, recurrent_dropout=0.3),
    # dropout to prevent overfitting
    Dropout(0.5),
    # dense to connect the previous output with current layer
    Dense(128, activation="relu"),
    # dropout to prevent overfitting
    Dropout(0.5),
    # this is output layer, with 3 class (0, 1, 2)
    Dense(3, activation="softmax"),
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',f1,precision, recall])


In [12]:
# checking the model parameters
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 26, 100)           1867100   
                                                                 
 lstm (LSTM)                 (None, 64)                42240     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 387       
                                                                 
Total params: 1,918,047
Trainable params: 1,918,047
Non-

In [13]:
# Train the model
model_history = model.fit(
    X_train,
    y_train,
    callbacks=[early_stopping],
    batch_size = 32,
    epochs=1000,
    validation_data=(X_test, y_test)
)

Epoch 1/1000
620/620 [==============================] - 20s 29ms/step - loss: 0.4318 - accuracy: 0.8523 - f1: 0.8356 - precision: 0.8592 - recall: 0.8167 - val_loss: 0.3283 - val_accuracy: 0.8864 - val_f1: 0.8851 - val_precision: 0.9080 - val_recall: 0.8641
Epoch 2/1000
620/620 [==============================] - 15s 25ms/step - loss: 0.2304 - accuracy: 0.9206 - f1: 0.9202 - precision: 0.9289 - recall: 0.9120 - val_loss: 0.3307 - val_accuracy: 0.8828 - val_f1: 0.8827 - val_precision: 0.8888 - val_recall: 0.8768
Epoch 3/1000
620/620 [==============================] - 15s 25ms/step - loss: 0.1510 - accuracy: 0.9458 - f1: 0.9459 - precision: 0.9495 - recall: 0.9425 - val_loss: 0.3628 - val_accuracy: 0.8866 - val_f1: 0.8870 - val_precision: 0.8908 - val_recall: 0.8834
Epoch 4/1000
620/620 [==============================] - 15s 24ms/step - loss: 0.1064 - accuracy: 0.9624 - f1: 0.9629 - precision: 0.9644 - recall: 0.9614 - val_loss: 0.4459 - val_accuracy: 0.8765 - val_f1: 0.8770 - val_precisi

In [14]:
# Print the best epoch (by vl loss)
best_epoch = early_stopping.stopped_epoch
print("Best Epoch:", best_epoch-4)

Best Epoch: 1


#### test 1 batch size 32
output_dim = 100
- loss: 0.4318
- accuracy: 0.8523
- f1: 0.8356
- precision: 0.8592
- recall: 0.8167
- val_loss: 0.3283
- val_accuracy: 0.8864
- val_f1: 0.8851
- val_precision: 0.9080
- val_recall: 0.8641

output_dim = 200
